# TartanAir V2 dataset

In this example, we provide more detailed IMU simulation for the TartanAir V2
dataset.  https://tartanair.org/

## Setup


## Specify a new IMU



In [ ]:
# -----------------------------------------------------------------------------
# Copyright (c) 2023-2025, Inertial Simulation LLC.
# All rights reserved.
# Do not use or redistribute without permission.
# Email: info@inertialsim.com
# -----------------------------------------------------------------------------
from inertialsim.sensors import Parameter
from inertialsim.sensors.imu import IMUSpecification

# Notes:
# 1. Specified with 8g accelerometer range.
# 2. Specified with 16 bit data resolution.
# 3. Scale factor min/max approximated as normal 3-sigma

epson_mg370: IMUSpecification = IMUSpecification()
"""Specification for an Epson M-G370PDT IMU.

Reference: Brief Sheet, Rev. 1.1, 2022.08.  
Link: https://global.epson.com/products_and_drivers/sensing_system/imu/m-g370pdt.html
"""  # noqa: W291
epson_mg370.manufacturer = "Epson"
epson_mg370.model = "M-G370"
epson_mg370.version = "PDT0"

epson_mg370.data_interface.sample_rate = Parameter(100, "Hz")
epson_mg370.data_interface.quantization = (
    Parameter(1 / 150, "deg/s/LSB"),
    Parameter(1e-3 / 4, "g/LSB"),
)

epson_mg370.gyro.input_limits.minimum = Parameter(-200.0, "deg/s")
epson_mg370.gyro.input_limits.maximum = Parameter(200.0, "deg/s")
epson_mg370.gyro.noise.random_walk = Parameter(0.03, "deg/sqrt(h)")
epson_mg370.gyro.noise.bias_instability = Parameter(0.8, "deg/h")
epson_mg370.gyro.bias.repeatability = Parameter(36, "deg/h")
epson_mg370.gyro.scale_factor.repeatability = Parameter(0.2 / 3, "%")
epson_mg370.gyro.misalignment.repeatability = Parameter(0.01, "deg")

epson_mg370.accelerometer.input_limits.minimum = Parameter(-8.0, "g")
epson_mg370.accelerometer.input_limits.maximum = Parameter(8.0, "g")
epson_mg370.accelerometer.noise.random_walk = Parameter(0.02, "m/s/sqrt(h)")
epson_mg370.accelerometer.noise.bias_instability = Parameter(24e-6, "g")
epson_mg370.accelerometer.bias.repeatability = Parameter(2e-3, "g")
epson_mg370.accelerometer.scale_factor.repeatability = Parameter(0.1 / 3, "%")
epson_mg370.accelerometer.misalignment.repeatability = Parameter(0.01, "deg")


## Load the data

In [ ]:
from inertialsim.examples import tartanair
from inertialsim.geometry import Vector
from inertialsim.devices.imu import epson_mg370
from inertialsim.sensors.imu import IMU, IMUModel

# Load the TartanAir V2 data.  Since we are just loading numpy arrays from file,
# we skip the built-in dataloader.  See:
# https://tartanair.org/examples.html#dataloader-example
imu_data, pose_data = tartanair.load(
    directory="tartanair_data", environment="Downtown", difficulty="hard", trajectory=6
)

# Ground-truth, 3-axis, orthogonal IMU input signals.
angular_rate = Vector.from_xyz(imu_data.angular_rate, imu_data.time)
specific_force = Vector.from_xyz(imu_data.specific_force, imu_data.time)

# Create the IMU model.  By default we simulate everything except delta-angle,
# delta-velocity outputs which are not supported by the Epson IMU.
model = IMUModel()
# model.data_interface.simulate_sample_rate = True
# model.data_interface.simulate_quantization = True
# model.noise.simulate_rate_random_walk = True
# model.misalignment.simulate_random = True
imu = IMU(model=model, specification=epson_mg370, rng=0)

# Simulate the IMU measurements
result = imu.simulate(angular_rate=angular_rate, specific_force=specific_force)
gyro_measurement = result.angular_rate
accelerometer_measrement = result.specific_force

# We can also extract the internal state of the simulator (noise streams, model
# settings, random number generator state, etc.)
sim_state = imu.state

## Analyze results

In [ ]:
# import numpy as np

# from inertialsim import plot
# from inertialsim.examples import tartanair
# from inertialsim.geometry import ExtendedPose, Rotation
# from inertialsim.geodesy import GlobalPose

# imu_data, pose_data = tartanair.load("tartanair_data", "Downtown", "easy", 0)

# attitude = Rotation.from_euler(
#     pose_data.attitude[:, [2, 1, 0]], sequence="ZYX", time=pose_data.time
# )
# local_pose = ExtendedPose.from_apv(attitude, pose_data.position, pose_data.velocity)
# pose = GlobalPose.from_local(pose=local_pose, gravity=[0, 0, 9.8])

# cam_time = np.genfromtxt(
#     "/home/mgeorge/code/examples/tartanair_data/Downtown/Data_easy/P000/imu/cam_time.txt"
# )
# cam_pose = np.genfromtxt(
#     "/home/mgeorge/code/examples/tartanair_data/Downtown/Data_easy/P000/pose_lcam_front.txt"
# )
# quaternion = cam_pose[:, 3:7]
# quaternion = quaternion / np.linalg.norm(quaternion, axis=1, keepdims=True)
# attitude2 = Rotation.from_quaternion(quaternion, time=cam_time, scalar_last=True)

In [ ]:
from inertialsim import plot
%matplotlib widget

# Plot the input
ar_plot = plot.TimeSeries(title="Angular Rate", ylabel="Angular Rate (rad/s)")
ar_plot.line(imu_data.time, imu_data.angular_rate)
ar_plot.legend(["X", "Y", "Z"])

gyro_plot = plot.TimeSeries(title="Angular Rate", ylabel="Angular Rate (rad/s)")
gyro_plot.line(result.angular_rate.time, result.angular_rate.data)
gyro_plot.legend(["X", "Y", "Z"])

diff_plot = plot.TimeSeries(title="Angular Rate", ylabel="Angular Rate (rad/s)")
diff_plot.line(result.angular_rate.time, result.angular_rate.data[:,:,0] - imu_data.angular_rate)
diff_plot.legend(["X", "Y", "Z"])

# ar2_plot = plot.TimeSeries(title="Angular Rate", ylabel="Angular Rate (rad/s)")
# ar2_plot.line(pose.time[:-1], pose.angular_rate())
# ar2_plot.line(imu_data.time, imu_data.angular_rate)
# ar2_plot.legend(["X", "Y", "Z"])


# # # Plot the input
# sf_plot = plot.TimeSeries(title="Specific Force", ylabel="Specific Force (m/s/s)")
# sf_plot.line(imu_data.time, imu_data.specific_force)
# sf_plot.legend(["X", "Y", "Z"])

# sf2_plot = plot.TimeSeries(title="Specific Force", ylabel="Specific Force (m/s/s)")
# sf2_plot.line(pose.time[:-1], pose.specific_force())
# sf2_plot.line(imu_data.time, imu_data.specific_force)
# sf2_plot.legend(["X", "Y", "Z"])

# # Plot the input
# att_plot = plot.TimeSeries(title="Attitude", ylabel="Attitude (rad)")
# att_plot.line(pose_data.time, pose_data.attitude)
# att_plot.legend(["X", "Y", "Z"])

# att_plot = plot.TimeSeries(title="Attitude", ylabel="Attitude (rad)")
# att_plot.line(attitude2.time, attitude2.as_euler(sequence="ZYX")[:,[2,1,0],:])
# att_plot.legend(["X", "Y", "Z"])

# pos_plot = plot.TimeSeries(title="Position", ylabel="Position (m)")
# pos_plot.line(pose_data.time, pose_data.position)
# pos_plot.legend(["X", "Y", "Z"])

# vel_plot = plot.TimeSeries(title="Velocity", ylabel="Velocity (m/s)")
# vel_plot.line(pose_data.time, pose_data.velocity)
# vel_plot.legend(["X", "Y", "Z"])

## Advanced Features

In [ ]:
imu_data, pose_data = tartanair.load("tartanair_data", "Downtown", "easy", 0)

attitude = Rotation.from_euler(
    pose_data.attitude[:, [2, 1, 0]], sequence="ZYX", time=pose_data.time
)
local_pose = ExtendedPose.from_apv(attitude, pose_data.position, pose_data.velocity)
pose = GlobalPose.from_local(pose=local_pose, gravity=[0, 0, 9.8])

cam_time = np.genfromtxt(
    "/home/mgeorge/code/examples/tartanair_data/Downtown/Data_easy/P000/imu/cam_time.txt"
)
cam_pose = np.genfromtxt(
    "/home/mgeorge/code/examples/tartanair_data/Downtown/Data_easy/P000/pose_lcam_front.txt"
)
quaternion = cam_pose[:, 3:7]
quaternion = quaternion / np.linalg.norm(quaternion, axis=1, keepdims=True)
attitude2 = Rotation.from_quaternion(quaternion, time=cam_time, scalar_last=True)

Epson IMU specifications are 36 deg/h gyro bias.  Earth rate is ~15 deg/h (360 per day) so very relevant.  6000 meters of altitude generates a change in gravity equivalent to the accelerometer bias.  Pittsburgh's gravity (at surface level) varies from the 9.8 constant by 0.2mg, approximately 10% of the bias term.